# Scraping the Federalist Papers from Yale's Website
#### I couldn't find the Federalist Papers all in one place
#### So I scraped them and put them in a pandas dataframe
#### Brief demonstration of web scraping with BeautifulSoup
#### I was planning on doing an EDA on the vocabulary of the three authors of the FP (Jay, Madison, Hamilton)
#### but I moved on to other projects 

In [1]:
import requests
import pandas as pd
import string
import re 
import nltk
import itertools
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from string import punctuation, digits

from federalist_methods import *

In [2]:
%%time 
url = 'https://avalon.law.yale.edu/18th_century/'

# accounting for inconsistent URLs on Yale's server
# I set up a for loop to gather each paper, each paper had its own URL
one_through_nine = [url+'fed0{0}.asp'.format(i) for i in range(1, 10)]
ten_through_eightysix = [url+'fed{0}.asp'.format(i) for i in range(10, 86)]
urls = one_through_nine + ten_through_eightysix

page_requests = [requests.get(url) for url in urls]
soups = [BeautifulSoup(page.text, 'html.parser') for page in page_requests]

CPU times: user 1.72 s, sys: 125 ms, total: 1.84 s
Wall time: 11.7 s


In [5]:
paper_total = [return_paper(i) for i in range(0, len(soups))]
author_total = [return_author(i) for i in range(0, len(soups))]
title_total = [return_title(i) for i in range(0, len(soups))]
cleaned_paper_total = [clean_paper(i) for i in range(0, len(soups))]

paper_s = pd.Series(paper_total)
author_s = pd.Series(author_total)
title_s = pd.Series(title_total)

In [6]:
cleaned_author = [re.sub('\r\n', '', item.strip()) for item in author_total]
cleaned2x_author = [re.sub('For the Independent Journal.', 'JAY', item.strip()) for item in cleaned_author]
flat_title_list = [title[0] for title in title_total]
cleaned_flat_title_list = [re.sub('The Federalist Papers : ', '', item) for item in flat_title_list]

In [7]:
federalist_df = pd.DataFrame(cleaned_paper_total, index=[cleaned2x_author, cleaned_flat_title_list])

In [8]:
federalist_df

0    \
HAMILTON No. 1              to the people of the state of new york   
JAY      No. 2              to the people of the state of new york   
         No. 3              to the people of the state of new york   
         No. 4              to the people of the state of new york   
         No. 5   it was remarked in the preceding paper that we...   
...                                                            ...   
HAMILTON No. 81             to the people of the state of new york   
         No. 82             to the people of the state of new york   
         No. 83             to the people of the state of new york   
         No. 84             to the people of the state of new york   
         No. 85             to the people of the state of new york   

                                                               1    \
HAMILTON No. 1   it is not however my design to dwell upon obse...   
JAY      No. 2   when the people of america reflect that they a...   
         No. 3   it is not a new observation that the people of...   
         No. 4   it is too true however disgraceful it may be t...   
         No. 5   this subject is copious and cannot easily be e...   
...                                                            ...   
HAMILTON No. 81  that there ought to be one court of supreme an...   
         No. 82  the erection of a new government whatever care...   
         No. 83  the objection to the plan of the convention wh...   
         No. 84  in the course of the foregoing review of the c...   
         No. 85  according to the formal division of the subjec...   

                                                               2    \
HAMILTON No. 1   i am well aware that it would be disingenuous ...   
JAY      No. 2   nothing is more certain than the indispensable...   
         No. 3   that consideration naturally tends to create g...   
         No. 4   these and a variety of other motives which aff...   
         No. 5   the history of great britain is the one with w...   
...                                                            ...   
HAMILTON No. 81  the reasons for it have been assigned in anoth...   
         No. 82  t is time only that can mature and perfect so ...   
         No. 83  the disingenuous form in which this objection ...   
         No. 84  there however remain a few which either did no...   
         No. 85  but these heads have been so fully anticipated...   

                                                               3    \
HAMILTON No. 1   candor will oblige us to admit that even such ...   
JAY      No. 2   it is well worthy of consideration therefore w...   
         No. 3   the more attentively i consider and investigat...   
         No. 4   but independent of these inducements to war wh...   
         No. 5   we may profit by their experience without payi...   
...                                                            ...   
HAMILTON No. 81  the only question that seems to have been rais...   
         No. 82  such questions accordingly have arisen upon th...   
         No. 83  the mere silence of the constitution in regard...   
         No. 84  these shall now be discussed but as the subjec...   
         No. 85  it is remarkable that the resemblance of the p...   

                                                               4    \
HAMILTON No. 1   so numerous indeed and so powerful are the cau...   
JAY      No. 2   it has until lately been a received and uncont...   
         No. 3   it is of high importance to the peace of ameri...   
         No. 4   with france and with britain we are rivals in ...   
         No. 5   although it seems obvious to common sense that...   
...                                                            ...   
HAMILTON No. 81  the same contradiction is observable in regard...   
         No. 82  the principal of these respect the situation o...   
         No. 83  to argue with respect to the latter would howe...   

In [8]:
joined_papers = []
for i in range(0, len(soups)):
    joined_papers.append([' '.join(item) for item in pd.Series(paper_total)[i]])

In [9]:
# a flatter dataframe 
df = pd.DataFrame(pd.Series(paper_total).explode())
df.reset_index(inplace=True)

In [12]:
df

,index,0
0,0,[to the people of the state of new york:]
1,0,"[it is not, however, my design to dwell upon o..."
2,0,"[and yet, however just these sentiments will b..."
3,0,"[i propose, in a series of papers, to discuss ..."
4,0,[in the progress of this discussion i shall en...
...,...,...
1115,84,"[this is not all., every constitution for the ..."
1116,84,"[but every amendment to the constitution, if o..."
1117,84,[in opposition to the probability of subsequen...
1118,84,"[if the foregoing argument is a fallacy, certa..."
